# Unit 6. Essential Deep Learning Tips & Tricks

# Exercise 2: Decreasing Overfitting with Weight Decay

In [7]:
%load_ext watermark
%watermark -p torch,lightning

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
torch    : 2.1.1
lightning: 2.1.2



In [8]:
import pytorch_lightning as L
import torch
import torch.nn.functional as F
import torchmetrics
from pytorch_lightning.loggers import CSVLogger

from shared_utilities import CustomDataModule, PyTorchMLP

In [9]:
num_epochs = 100

In [10]:
class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate, cosine_t_max):
        super().__init__()

        self.learning_rate = learning_rate
        self.cosine_t_max = cosine_t_max
        self.model = model

        self.save_hyperparameters(ignore=["model"])

        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

    def forward(self, x):
        return self.model(x)

    def _shared_step(self, batch):
        features, true_labels = batch
        logits = self(features)

        loss = F.cross_entropy(logits, true_labels)
        predicted_labels = torch.argmax(logits, dim=1)
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("train_loss", loss)
        self.train_acc(predicted_labels, true_labels)
        self.log(
            "train_acc", self.train_acc, prog_bar=True, on_epoch=True, on_step=False
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("val_loss", loss, prog_bar=True)
        self.val_acc(predicted_labels, true_labels)
        self.log("val_acc", self.val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.test_acc(predicted_labels, true_labels)
        self.log("test_acc", self.test_acc)

    def configure_optimizers(self):
        
        ##################################################################################
        ### YOUR TASK: Replace Adam with AdamW and experiment with the weight_decay param
        ##################################################################################
        
        opt = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=self.cosine_t_max)

        return [opt], [sch]

In [11]:
L.seed_everything(123)
dm = CustomDataModule()

pytorch_model = PyTorchMLP(num_features=100, num_classes=2)
lightning_model = LightningModel(model=pytorch_model, learning_rate=0.005,
                                 cosine_t_max=num_epochs)

trainer = L.Trainer(
    max_epochs=num_epochs,
    accelerator="cpu",
    devices="auto",
    logger=CSVLogger(save_dir="logs/", name="my-model"),
    deterministic=True,
)

Global seed set to 123
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/galimalki/anaconda3/envs/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/setup.py:201: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.


In [ ]:
trainer.fit(model=lightning_model, datamodule=dm)

Missing logger folder: logs/my-model

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | PyTorchMLP         | 15.3 K
1 | train_acc | MulticlassAccuracy | 0     
2 | val_acc   | MulticlassAccuracy | 0     
3 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
15.3 K    Trainable params
0         Non-trainable params
15.3 K    Total params
0.061     Total estimated model params size (MB)


/Users/galimalki/anaconda3/envs/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
/Users/galimalki/anaconda3/envs/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


Epoch 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 450/450 [00:01<00:00, 356.50it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████████████████████████████████████████████████████████████████| 450/450 [00:01<00:00, 392.26it/s, v_num=0, val_loss=0.514, val_acc=0.751, train_acc=0.725]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████████████████████████████████████████████████████████████████| 450/450 [00:01<00:00, 363.36it/s, v_num=0, val_loss=0.469, val_acc=0.792, train_acc=0.783]
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████████████████████████████████████████████████████████████████| 450/450 [00:01<00:00, 280.73it/s, v_num=0, val_loss=0.450, val_acc=0.808, train_acc=0.817]
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████████████████████████████████████████████████████████████████| 450/450 [00:01<00:00, 275.10it/s, v_num=0, val_loss=0.409, val_acc=0.829, train_ac

In [ ]:
### Plot
import matplotlib.pyplot as plt
import pandas as pd

metrics = pd.read_csv(f"{trainer.logger.log_dir}/metrics.csv")

aggreg_metrics = []
agg_col = "epoch"
for i, dfg in metrics.groupby(agg_col):
    agg = dict(dfg.mean())
    agg[agg_col] = i
    aggreg_metrics.append(agg)

df_metrics = pd.DataFrame(aggreg_metrics)
df_metrics[["train_loss", "val_loss"]].plot(
    grid=True, legend=True, xlabel="Epoch", ylabel="Loss"
)

#plt.savefig("suggest_loss.pdf")

df_metrics[["train_acc", "val_acc"]].plot(
    grid=True, legend=True, xlabel="Epoch", ylabel="ACC"
)

plt.ylim([0.6, 1])
#plt.savefig("suggest_acc.pdf")
plt.show()

In [ ]:
trainer.test(model=lightning_model, datamodule=dm)